In [1]:
import pandas as pd
import numpy as np
import googlemaps
from datetime import datetime
import acquire_r
from env import google_key
from googlemaps import convert
import wrangle_zillow_r

In [2]:
gmaps = googlemaps.Client(key=google_key)

In [150]:
df = acquire_r.get_zillow_data()

In [151]:
df1 = df[['latitude', 'longitude', 'regionidzip']]

In [152]:
df1 = df1.dropna()

In [153]:
df1 = df1.drop_duplicates('regionidzip', keep='first')
df2 = df1.copy()

In [154]:
df1 = df1.drop(columns = 'regionidzip')

In [158]:
df1 = df1/1000000
df1.head(3)

,latitude,longitude
54984,0.000034,-0.000119
2912,0.000034,-0.000119
52774,0.000034,-0.000119


In [254]:
df2

,index,latitude,longitude,regionidzip
0,54984,34222559.0,-118617387.0,96339.0
1,2912,34238190.0,-118638614.0,96346.0
2,52774,34207150.0,-118630445.0,96342.0
3,2803,34207627.0,-118606039.0,96338.0
4,23096,34180300.0,-118666006.0,96389.0
...,...,...,...,...
384,18018,34312328.0,-118907683.0,97097.0
385,30260,34231419.0,-118870928.0,96383.0
386,70083,34274500.0,-118810000.0,97118.0
387,54808,34279241.0,-118729175.0,97116.0


In [30]:
df1
records = df1.to_records(index=False)
list2 = list(records)

In [62]:
def zip_code(list):
    geocode_results = []
    for i in range (0, len(list)):
        geocode_results.append(gmaps.reverse_geocode(list[i])[1]['formatted_address'])
    return geocode_results

In [63]:
new_zip = zip_code(list2)

In [64]:
new_zip

['8394 Capistrano Ave, Canoga Park, CA 91304, USA',
 '9243 Johnell Rd, Chatsworth, CA 91311, USA',
 '23112 Covello St, West Hills, CA 91307, USA',
 '7541 Topanga Canyon Blvd, Canoga Park, CA 91303, USA',
 '5963 Vista De La Luz, Woodland Hills, CA 91367, USA',
 '5900 Gleam Ct, Agoura Hills, CA 91301, USA',
 '5890 Jed Smith Rd, Hidden Hills, CA 91302, USA',
 '30841 Monaco Ct, Westlake Village, CA 91362, USA',
 '30860 Agoura Rd, Westlake Village, CA 91361, USA',
 '4626 Larkwood Ave, Woodland Hills, CA 91364, USA',
 '8195 White Oak Ave, Reseda, CA 91335, USA',
 '13031 Hesperia Ave, Reseda, CA 91335, USA',
 '19533 Roscoe Blvd, Northridge, CA 91324, USA',
 '19960 Cantara St, Canoga Park, CA 91306, USA',
 '6375 Enfield Ave, Encino, CA 91316, USA',
 '6020 Melvin Ave, Tarzana, CA 91356, USA',
 '7430 Louise Ave, Van Nuys, CA 91406, USA',
 '8126 Noble Ave, Van Nuys, CA 91402, USA',
 '15115 Saticoy St, Van Nuys, CA 91405, USA',
 '15138 Haynes St, Van Nuys, CA 91411, USA',
 '14103 Hamlin St, Van Nu

In [83]:
new_zip[0].split()[-2][0:5]

'91304'

In [116]:
zip_list = []
for i in range (0, len(new_zip)):
    zip_list.append(new_zip[i].split()[-2][0:5])

In [200]:
df3 = pd.DataFrame(zip_list)

In [251]:
df3

,0
0,91304
1,91311
2,91307
3,91303
4,91367
...,...
384,93021
385,91360
386,93065
387,93063


In [192]:
df2 = df2.reset_index()

In [194]:
zip_codes = pd.concat([df2, df3], axis =1)

In [198]:
zip_codes = zip_codes.set_index('index').rename(columns = {0:'new_zip'})

In [199]:
zip_codes

,latitude,longitude,regionidzip,new_zip
index,,,,
54984,34222559.0,-118617387.0,96339.0,91304
2912,34238190.0,-118638614.0,96346.0,91311
52774,34207150.0,-118630445.0,96342.0,91307
2803,34207627.0,-118606039.0,96338.0,91303
23096,34180300.0,-118666006.0,96389.0,91367
...,...,...,...,...
18018,34312328.0,-118907683.0,97097.0,93021
30260,34231419.0,-118870928.0,96383.0,91360
70083,34274500.0,-118810000.0,97118.0,93065


In [202]:
zip_codes.to_csv('zip_codes.csv')

In [2]:
zip_codes = pd.read_csv('zip_codes.csv')
zip_codes['new_zip'] = zip_codes.new_zip.astype(float)

In [3]:
zip_codes['new_zip'] = zip_codes.new_zip.astype(int)

In [205]:
income = pd.read_clipboard()

In [215]:
income
income['Zip'] = income['Zip'].astype(int)

In [222]:
zip_income = pd.merge(zip_codes, income, how = 'left', left_on = 'new_zip', right_on = 'Zip')

In [224]:
zip_income.to_csv('zipcode_income.csv')

In [2]:
zillow = pd.read_csv('zillow.csv')

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (33,35,62,63,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
zillow = zillow[zillow.propertylandusetypeid.isin(single_use)]

In [4]:
zillow.propertylandusetypeid.value_counts()

261.0    52439
266.0    19341
263.0       74
264.0        6
Name: propertylandusetypeid, dtype: int64

In [5]:
zillow = zillow[(zillow.bedroomcnt > 0) & (zillow.bathroomcnt > 0)]

In [6]:
zillow.unitcnt.value_counts()

1.0    45166
2.0       32
3.0        3
6.0        1
Name: unitcnt, dtype: int64

In [7]:
zillow['county'] = np.where(zillow.fips == 6037, 'Los_Angeles',
                            np.where(zillow.fips == 6059, 'Orange', 'Ventura'))

In [8]:
zillow['regionidzip'] = zillow['regionidzip'].replace(399675, 99675)

In [12]:
zillow1.census_tractandblock.isnull().sum()

0

In [10]:
zillow1 = wrangle_zillow_r.zillow_impute_knn(zillow)

In [11]:
zillow1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73752 entries, 0 to 73751
Data columns (total 70 columns):
id                              73752 non-null int64
parcelid                        73752 non-null int64
airconditioningtypeid           23627 non-null float64
architecturalstyletypeid        206 non-null float64
basementsqft                    47 non-null float64
bathroomcnt                     73752 non-null float64
bedroomcnt                      73752 non-null float64
buildingclasstypeid             0 non-null float64
buildingqualitytypeid           46565 non-null float64
calculatedbathnbr               73736 non-null float64
decktypeid                      587 non-null float64
finishedfloor1squarefeet        5818 non-null float64
calculatedfinishedsquarefeet    73742 non-null float64
finishedsquarefeet12            73575 non-null float64
finishedsquarefeet13            0 non-null float64
finishedsquarefeet15            9 non-null float64
finishedsquarefeet50            581

In [23]:
pd.options.display.float_format = '{:.1f}'.format
zillow1.census_tractandblock.astype(object)

0       60371132321007.0
1       60371132321007.0
2       60371132321007.0
3       60371132321008.0
4       60371132321008.0
              ...       
73747   60371132321007.0
73748   60371132321007.0
73749   60371132321007.0
73750   60371132321007.0
73751   60371132321007.0
Name: census_tractandblock, Length: 73752, dtype: object